In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
%matplotlib inline
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import shuffle
plt.rcParams["figure.figsize"] = (12, 9) # (w, h)

##### we will fetch data from both 'case_study_actions_train.csv' and 'case_study_bookings_train.csv'. 
##### Because, the most significant action should be the one which yeilds to maximum bookings.

In [2]:
action_train = pd.read_csv('case_study_actions_train.csv', sep='\t')
book_train =  pd.read_csv('case_study_bookings_train.csv', sep='\t')

In [3]:
action_train.head()

,ymd,user_id,session_id,action_id,reference,step
0,20170423,388309106223940,3052767322364990735,8001,1323836,1
1,20170410,452426828488840,1022778951418899936,2116,929835,1
2,20170410,452426828488840,1022778951418899936,6999,0,2
3,20170410,452426828488840,1022778951418899936,2116,929835,3
4,20170410,452426828488840,1022778951418899936,2503,1,4


In [34]:
# Checking the shape of action_train
action_train.shape

(5862863, 6)

In [4]:
book_train.head()

,ymd,user_id,session_id,referer_code,is_app,agent_id,traffic_type,has_booking
0,20170423,388309106223940,3052767322364990735,0,0,2,1,0
1,20170410,452426828488840,1022778951418899936,0,0,10,2,0
2,20170415,452426828488840,4191504489082712531,0,0,10,2,0
3,20170406,819438352219100,4560227804862289210,1,0,1,1,0
4,20170407,1113732603712480,4115013282086590434,0,0,9,2,0


In [35]:
# Checking the shape of book_train
book_train.shape

(307677, 8)

#### Merging both dataframe on basis of 'ymd','user_id' and 'session_id'

In [36]:
result = pd.merge(action_train,
                 book_train[['ymd','user_id','session_id','has_booking']],
                 on=['ymd','user_id','session_id'])

In [37]:
#Checking the new dataframe
result.head()

,ymd,user_id,session_id,action_id,reference,step,has_booking
0,20170423,388309106223940,3052767322364990735,8001,1323836,1,0
1,20170410,452426828488840,1022778951418899936,2116,929835,1,0
2,20170410,452426828488840,1022778951418899936,6999,0,2,0
3,20170410,452426828488840,1022778951418899936,2116,929835,3,0
4,20170410,452426828488840,1022778951418899936,2503,1,4,0


In [38]:
#Extracting the relevant columns from the new dataframe.
action_id_has_booking = result[['action_id', 'has_booking']]
action_id_has_booking.head()

,action_id,has_booking
0,8001,0
1,2116,0
2,6999,0
3,2116,0
4,2503,0


In [39]:
# Checking frequency of every action_id
action_id_has_booking['action_id'].value_counts()

2142    1150642
2113     689437
8001     343534
2116     278855
6999     253549
2111     180022
2371     156456
2119     145060
2358     138146
2503     135171
2166     135062
2114     124381
2351     117550
2350     117257
2155     106085
2115      98358
2146      95208
2100      90693
2306      86732
2175      84461
2357      75856
2773      68983
2710      41487
2792      40479
2702      40135
2765      39455
2706      37180
2501      34692
2123      30921
2121      28349
         ...   
2153         92
2186         88
2785         78
2854         73
2856         64
2292         59
2327         56
2865         55
2877         48
2855         41
2878         40
2394         39
2705         37
2859         35
8006         35
2858         27
2882         24
2382         21
2715         20
2148         15
2191         11
2506          9
2896          8
2861          6
2149          5
2172          3
2269          2
2225          2
8091          2
2890          1
Name: action_id, Length:

In [40]:
# Checking frequency of both categories of 'has_booking'
action_id_has_booking['has_booking'].value_counts()

0    5100554
1     762309
Name: has_booking, dtype: int64

### Generating contingency table

In [41]:
contingency_table = pd.crosstab(
    action_id_has_booking['action_id'],
    action_id_has_booking['has_booking'],
    margins = True
)
contingency_table

has_booking,0,1,All
action_id,,,
29,91,11,102
2100,77395,13298,90693
2111,169148,10874,180022
2113,613847,75590,689437
2114,108592,15789,124381
2115,81281,17077,98358
2116,257001,21854,278855
2119,122871,22189,145060
2121,23109,5240,28349


In [28]:
# Removing the last row with "All"
main_table = contingency_table[1:210]

In [30]:
# Removing the last column with "All"
main_table = main_table.iloc[:,0:2]

In [42]:
main_table.head()

has_booking,0,1
action_id,,
2100,77395,13298
2111,169148,10874
2113,613847,75590
2114,108592,15789
2115,81281,17077


## Extracting the 'action_id' with maximum #has_booking

In [43]:
main_table[1].argmax()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  """Entry point for launching an IPython kernel.


2142

In [44]:
# Writing the main_table to csv
main_table.to_csv("contingency_table.csv", sep=',', encoding='utf-8', index=True)

# Hence, the most significant action should be : 2142